In [0]:
#importing all the required packages
import numpy as np
import pandas as pd
import nltk as nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import statistics as s
from sklearn.metrics import confusion_matrix 
from scipy.sparse import hstack

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#importing dataset that is preprocessed
df=pd.read_csv('/content/drive/My Drive/AIt/job_data.csv')

In [5]:
df.head()

,Unnamed: 0,job_data,telecommuting,has_company_logo,has_questions,fraudulent,Contract,Full-time,Other,Part-time,Temporary,Information Technology,Management,Marketing and Finance,Other.1,Sales
0,0,market intern we re food52 and we ve creat a g...,0,1,0,0,0,0,1,0,0,0,0,1,0,0
1,1,custom servic cloud video product 90 second th...,0,1,0,0,0,1,0,0,0,0,0,0,1,0
2,2,commiss machineri assist cma valor servic prov...,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,3,account execut washington dc our passion for i...,0,1,0,0,0,1,0,0,0,0,0,0,0,1
4,4,bill review manag spotsourc solut llc is a glo...,0,1,1,0,0,1,0,0,0,0,0,0,1,0


In [0]:
#Dropping the Un-named column
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [0]:
# copy the data frame into a new variable to use it for horizontal stack
new_df=df.copy()
new_df.drop(['job_data','fraudulent'],axis=1,inplace=True)

In [0]:
#Preparing the data for train test split
x=df.drop('fraudulent',axis=1)
y=df['fraudulent']

In [0]:
#splitting the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [0]:
#reset the custom index generated during train, test split
X_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [11]:
#check the head of the train data
X_train.head()

,job_data,telecommuting,has_company_logo,has_questions,Contract,Full-time,Other,Part-time,Temporary,Information Technology,Management,Marketing and Finance,Other.1,Sales
0,administr assist katalystfp is a specialti fin...,0,1,1,0,1,0,0,0,0,1,0,0,0
1,client success manag we live in an increasingl...,0,1,1,0,1,0,0,0,0,0,0,0,1
2,regist nurs rn spotsourc solut llc is a global...,1,1,1,0,1,0,0,0,0,0,0,1,0
3,applic support engin industri color softwar is...,0,1,0,0,1,0,0,0,1,0,0,0,0
4,frontend ninja and web develop hello worldtale...,0,1,1,0,1,0,0,0,1,0,0,0,0


In [12]:
#Doing K fold Cross Validation on the training set. We use 5 folds so n_splits=5
folds = StratifiedKFold(n_splits=5, random_state=0)
i=0
#create a list for storing accuracy reports and scores
acc_report=[]
acc_score=[]
for train, test in folds.split(X_train, y_train):
    print(train)
    print(test)
    #taking only text data from training data into another variable to perform TF-idf vectorization
    x_train1, x_test1 = X_train['job_data'][train], X_train['job_data'][test]
    #taking the test data into another variable 
    y_train1, y_test1 = y_train[train], y_train[test]
    i=i+1
    #Using TF-IDF vectorizer to generate a TF-IDF matrix with ngram range (1,2)
    tfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
    x_train1 = tfidf.fit_transform(x_train1)
    x_test1 = tfidf.transform(x_test1)
    print("tfidf"+str(i))
    
    #Using the copy of the dataframe with no job_data to access the other features of the dataset
    rx_train=new_df.iloc[train]
    rx_test=new_df.iloc[test]

    #Use hstack to horizontally stack the job_data Tf-idf matrix to other features of the dataset
    xtrain_data=hstack( [x_train1,rx_train] )
    xtest_data=hstack( [x_test1,rx_test] )

    #Perform Naive Bayes classification to make predictions on the train data and check how model works on our dataset.
    clf = MultinomialNB(alpha=1.0)
    clf.fit(xtrain_data, y_train1)
    y_pred = clf.predict(xtest_data)
    scores = accuracy_score(y_test1, y_pred)
    report=classification_report(y_test1,y_pred)
    #use append() to store all the 5 reports and scores in the created list
    acc_report.append(report)
    acc_score.append(scores)
    print(scores)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[ 2855  2856  2857 ... 14300 14301 14302]
[   0    1    2 ... 2976 3015 3037]
tfidf1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.9510660608178958
[    0     1     2 ... 14300 14301 14302]
[2855 2856 2857 ... 5719 5720 5721]
tfidf2


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.9510660608178958
[    0     1     2 ... 14300 14301 14302]
[ 8480  8488  8501 ... 11455 11456 11457]
tfidf4


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.9513986013986014
[    0     1     2 ... 11455 11456 11457]
[11163 11174 11184 ... 14300 14301 14302]
tfidf5
0.9513986013986014


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#mean of validation score
  print(s.mean(acc_score))

0.9515486540722472


In [0]:
#Now using TF-IDF vectorizer to generate TF-IDF matrix for both X_train and x_test with ngram range (1,2)
tfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
x_train=tfidf.fit_transform(X_train['job_data'])
x_test=tfidf.transform(X_test['job_data'])

In [0]:
# The shape of X_train will tell us how many ngrams have been generated
x_train.shape

(14303, 778233)

In [0]:
#Drop job_data from train and test set for horizontal stacking of job_data's TF-IDF matrix
X_train.drop('job_data',axis=1,inplace=True)
X_test.drop('job_data',axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
#Use hstack to horizontal stack the TF-IDF matrix to the other features
train_data=hstack( [X_train,x_train] )
test_data=hstack( [X_test,x_test] )

In [0]:
#using Naive Bayes with smoothing factor value as 1 and make predictionb on test data
clf = MultinomialNB(alpha=1.0)
clf.fit(train_data,y_train)
pred = clf.predict(test_data)
scores = accuracy_score(y_test, pred)
report=classification_report(y_test, pred)
# Print the accuracy report, scores and confusion matrix
print("Naive Bayes Results will all features:")
print("Accuracy:","\n",scores)
print("Accuracy report:","\n",report)
print("Confusion Matrix:","\n",confusion_matrix(y_test, pred))

Naive Bayes Results will all features:
Accuracy: 
 0.9519015659955258
Accuracy report: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      3404
           1       0.00      0.00      0.00       172

    accuracy                           0.95      3576
   macro avg       0.48      0.50      0.49      3576
weighted avg       0.91      0.95      0.93      3576

Confusion Matrix: 
 [[3404    0]
 [ 172    0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#Using Logistic Regression with default parameters to make predictions on test data
classifier = LogisticRegression()
classifier.fit(train_data , y_train)
prediction=classifier.predict(test_data)
model_score = accuracy_score(y_test, prediction)
class_report=classification_report(y_test, prediction)
# Print the accuracy report, scores and confusion matrix
print("Logistic Regression using all the features")
print("Accuracy:","\n", model_score)
print("Accuracy report:","\n", report)
print("Confusion Matrix:","\n", confusion_matrix(y_test, prediction))

Logistic Regression using all the features
Accuracy: 
 0.9725950782997763
Accuracy report: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      3404
           1       0.00      0.00      0.00       172

    accuracy                           0.95      3576
   macro avg       0.48      0.50      0.49      3576
weighted avg       0.91      0.95      0.93      3576

Confusion Matrix: 
 [[3401    3]
 [  95   77]]


In [0]:
# Using Logistic regression with parameters Penalty 'l2', solver 'lbfgs' and multi_class as 'ovr'
classifier = LogisticRegression(penalty='l2',solver='lbfgs',multi_class='ovr')
classifier.fit(train_data , y_train)
prediction=classifier.predict(test_data)
model_score = accuracy_score(y_test, prediction)
class_report=classification_report(y_test, prediction)
# Print the accuracy report, scores and confusion matrix
print("Logistic Regression using all the features:")
print("Accuracy:","\n", model_score)
print("Accuracy report:","\n", report)
print("Confusion Matrix:","\n", confusion_matrix(y_test, prediction))

Logistic Regression using all the features:
Accuracy: 
 0.9725950782997763
Accuracy report: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      3404
           1       0.00      0.00      0.00       172

    accuracy                           0.95      3576
   macro avg       0.48      0.50      0.49      3576
weighted avg       0.91      0.95      0.93      3576

Confusion Matrix: 
 [[3401    3]
 [  95   77]]


In [0]:
#Using logistic regression as before with parameters Penalty 'l2', solver 'lbfgs' and multi_class as 'ovr'
#Now set the parameter class_weight as 'balanced'
classifier = LogisticRegression(penalty='l2',solver='lbfgs',multi_class='ovr',class_weight='balanced')
classifier.fit(train_data , y_train)
prediction=classifier.predict(test_data)
model_score = accuracy_score(y_test, prediction)
class_report=classification_report(y_test, prediction)
# Print the accuracy report, scores and confusion matrix
print("Logistic Regression using all the features:")
print("Accuracy:","\n", model_score)
print("Accuracy report:","\n", report)
print("Confusion Matrix:","\n", confusion_matrix(y_test, prediction))

Logistic Regression using all the features:
Accuracy: 
 0.9611297539149888
Accuracy report: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      3404
           1       0.00      0.00      0.00       172

    accuracy                           0.95      3576
   macro avg       0.48      0.50      0.49      3576
weighted avg       0.91      0.95      0.93      3576

Confusion Matrix: 
 [[3281  123]
 [  16  156]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
